In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
from utils_ray import scale_columns, create_windows, append_segments
 

Using device: cuda


In [27]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = out[:, -1, :]  # Take the output of the last time step
        out = self.fc(out)
        return out

def create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size=32):
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

def evaluate_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
    accuracy = correct_predictions / total_predictions
    return running_loss / len(dataloader.dataset), accuracy

def train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience):
    best_val_loss = float('inf')
    best_model_wts = None
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = model.state_dict().copy()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print('Early stopping triggered')
            break

    model.load_state_dict(best_model_wts)
    return model

def test_model(model, test_loader):
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
    return np.array(all_predictions)


In [28]:
window_sizes = [10, 20]
num_classes = 5  # Update this with your actual number of classes
input_size = 3  # Update this with your actual number of features
hidden_size = 50
num_epochs = 100
patience = 3
batch_size = 32

df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)
df_copy = df.copy()
df_copy['activity'].replace({77: 0}, inplace=True)
# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

for window_size in window_sizes:
    X, y = create_windows(df_copy, window_size)
    
    # Splitting the dataset into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

    model = LSTMModel(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience)

    predictions = test_model(model, test_loader)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, predictions)
    print(f"WINDOW SIZE = {window_size}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # Print a classification report
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))

Epoch 1/100, Train Loss: 0.2484, Val Loss: 0.2184, Val Accuracy: 0.9290
Epoch 2/100, Train Loss: 0.2030, Val Loss: 0.1947, Val Accuracy: 0.9375
Epoch 3/100, Train Loss: 0.1841, Val Loss: 0.1773, Val Accuracy: 0.9422
Epoch 4/100, Train Loss: 0.1712, Val Loss: 0.1681, Val Accuracy: 0.9443
Epoch 5/100, Train Loss: 0.1603, Val Loss: 0.1578, Val Accuracy: 0.9475
Epoch 6/100, Train Loss: 0.1514, Val Loss: 0.1505, Val Accuracy: 0.9504
Epoch 7/100, Train Loss: 0.1441, Val Loss: 0.1456, Val Accuracy: 0.9510
Epoch 8/100, Train Loss: 0.1367, Val Loss: 0.1400, Val Accuracy: 0.9522
Epoch 9/100, Train Loss: 0.1307, Val Loss: 0.1359, Val Accuracy: 0.9543
Epoch 10/100, Train Loss: 0.1254, Val Loss: 0.1315, Val Accuracy: 0.9549
Epoch 11/100, Train Loss: 0.1200, Val Loss: 0.1254, Val Accuracy: 0.9570
Epoch 12/100, Train Loss: 0.1154, Val Loss: 0.1284, Val Accuracy: 0.9553
Epoch 13/100, Train Loss: 0.1108, Val Loss: 0.1206, Val Accuracy: 0.9586
Epoch 14/100, Train Loss: 0.1069, Val Loss: 0.1171, Val Accu

In [30]:
window_sizes = [30, 40, 60]
num_classes = 5  # Update this with your actual number of classes
input_size = 3  # Update this with your actual number of features
hidden_size = 50
num_epochs = 100
patience = 3
batch_size = 32

df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)
df_copy = df.copy()
df_copy['activity'].replace({77: 0}, inplace=True)
# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

for window_size in window_sizes:
    X, y = create_windows(df_copy, window_size)
    
    # Splitting the dataset into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

    model = LSTMModel(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience)

    predictions = test_model(model, test_loader)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, predictions)
    print(f"WINDOW SIZE = {window_size}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # Print a classification report
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))

Epoch 1/100, Train Loss: 0.2235, Val Loss: 0.1952, Val Accuracy: 0.9397
Epoch 2/100, Train Loss: 0.1427, Val Loss: 0.1238, Val Accuracy: 0.9586
Epoch 3/100, Train Loss: 0.1091, Val Loss: 0.1023, Val Accuracy: 0.9653
Epoch 4/100, Train Loss: 0.0897, Val Loss: 0.0842, Val Accuracy: 0.9708
Epoch 5/100, Train Loss: 0.0733, Val Loss: 0.0672, Val Accuracy: 0.9771
Epoch 6/100, Train Loss: 0.0605, Val Loss: 0.0562, Val Accuracy: 0.9814
Epoch 7/100, Train Loss: 0.0512, Val Loss: 0.0477, Val Accuracy: 0.9841
Epoch 8/100, Train Loss: 0.0440, Val Loss: 0.0439, Val Accuracy: 0.9856
Epoch 9/100, Train Loss: 0.0379, Val Loss: 0.0407, Val Accuracy: 0.9861
Epoch 10/100, Train Loss: 0.0333, Val Loss: 0.0353, Val Accuracy: 0.9879
Epoch 11/100, Train Loss: 0.0296, Val Loss: 0.0364, Val Accuracy: 0.9873
Epoch 12/100, Train Loss: 0.0264, Val Loss: 0.0294, Val Accuracy: 0.9901
Epoch 13/100, Train Loss: 0.0239, Val Loss: 0.0262, Val Accuracy: 0.9916
Epoch 14/100, Train Loss: 0.0220, Val Loss: 0.0247, Val Accu

In [41]:
window_sizes = [200, 300]
num_classes = 5  # Update this with your actual number of classes
input_size = 3  # Update this with your actual number of features
hidden_size = 50
num_epochs = 100
patience =3
batch_size = 32

df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)
df_copy = df.copy()
df_copy['activity'].replace({77: 0}, inplace=True)
# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

for window_size in window_sizes:
    X, y = create_windows(df_copy, window_size)
    
    # Splitting the dataset into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

    model = LSTMModel(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience)

    predictions = test_model(model, test_loader)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, predictions)
    print(f"WINDOW SIZE = {window_size}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # Print a classification report
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))
    save_path = f"./submission_model_new_split_window_size_{window_size}.pth"
    torch.save(model, save_path)
    print(f"Model saved to {save_path}")

Epoch 1/100, Train Loss: 0.2527, Val Loss: 0.1825, Val Accuracy: 0.9413
Epoch 2/100, Train Loss: 0.1485, Val Loss: 0.0933, Val Accuracy: 0.9713
Epoch 3/100, Train Loss: 0.0682, Val Loss: 0.0452, Val Accuracy: 0.9845
Epoch 4/100, Train Loss: 0.0406, Val Loss: 0.0344, Val Accuracy: 0.9898
Epoch 5/100, Train Loss: 0.0283, Val Loss: 0.0506, Val Accuracy: 0.9846
Epoch 6/100, Train Loss: 0.0245, Val Loss: 0.0240, Val Accuracy: 0.9924
Epoch 7/100, Train Loss: 0.0218, Val Loss: 0.0156, Val Accuracy: 0.9956
Epoch 8/100, Train Loss: 0.0146, Val Loss: 0.0107, Val Accuracy: 0.9966
Epoch 9/100, Train Loss: 0.0119, Val Loss: 0.0059, Val Accuracy: 0.9983
Epoch 10/100, Train Loss: 0.0111, Val Loss: 0.0148, Val Accuracy: 0.9970
Epoch 11/100, Train Loss: 0.0098, Val Loss: 0.0090, Val Accuracy: 0.9979
Epoch 12/100, Train Loss: 0.0078, Val Loss: 0.0054, Val Accuracy: 0.9984
Epoch 13/100, Train Loss: 0.0065, Val Loss: 0.0096, Val Accuracy: 0.9972
Epoch 14/100, Train Loss: 0.0062, Val Loss: 0.0038, Val Accu

In [39]:
save_path = f"./submission_model_new_split_window_size_100.pth"
torch.save(model, save_path)
print(f"Model saved to {save_path}")

Model saved to ./submission_model_new_split_window_size_100.pth


In [34]:
window_sizes = [50]
num_classes = 5  # Update this with your actual number of classes
input_size = 3  # Update this with your actual number of features
hidden_size = 50
num_epochs = 100
patience = 5
batch_size = 32

df = pd.read_csv("movementSensorData.csv")
df = df.rename(columns={'Unnamed: 0': 'time_ms'})
df = append_segments(df)
df_copy = df.copy()
df_copy['activity'].replace({77: 0}, inplace=True)
# Normalize specific columns
columns_to_normalize = ['lw_x', 'lw_y', 'lw_z']
scaler, df_copy = scale_columns(df_copy, columns_to_normalize)

for window_size in window_sizes:
    X, y = create_windows(df_copy, window_size)
    
    # Splitting the dataset into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

    model = LSTMModel(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience)

    predictions = test_model(model, test_loader)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, predictions)
    print(f"WINDOW SIZE = {window_size}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # Print a classification report
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))

Epoch 1/100, Train Loss: 0.2041, Val Loss: 0.1345, Val Accuracy: 0.9567
Epoch 2/100, Train Loss: 0.1085, Val Loss: 0.0841, Val Accuracy: 0.9723
Epoch 3/100, Train Loss: 0.0706, Val Loss: 0.0585, Val Accuracy: 0.9805
Epoch 4/100, Train Loss: 0.0499, Val Loss: 0.0538, Val Accuracy: 0.9822
Epoch 5/100, Train Loss: 0.0363, Val Loss: 0.0288, Val Accuracy: 0.9907
Epoch 6/100, Train Loss: 0.0270, Val Loss: 0.0206, Val Accuracy: 0.9935
Epoch 7/100, Train Loss: 0.0201, Val Loss: 0.0221, Val Accuracy: 0.9933
Epoch 8/100, Train Loss: 0.0162, Val Loss: 0.0200, Val Accuracy: 0.9933
Epoch 9/100, Train Loss: 0.0141, Val Loss: 0.0150, Val Accuracy: 0.9950
Epoch 10/100, Train Loss: 0.0118, Val Loss: 0.0076, Val Accuracy: 0.9978
Epoch 11/100, Train Loss: 0.0113, Val Loss: 0.0148, Val Accuracy: 0.9948
Epoch 12/100, Train Loss: 0.0094, Val Loss: 0.0103, Val Accuracy: 0.9970
Epoch 13/100, Train Loss: 0.0091, Val Loss: 0.0055, Val Accuracy: 0.9986
Epoch 14/100, Train Loss: 0.0080, Val Loss: 0.0101, Val Accu

In [35]:
save_path = f"./submission_model_windowsize_50.pth"
torch.save(model, save_path)
print(f"Model saved to {save_path}")

Model saved to ./submission_model_windowsize_50.pth
